In [ ]:
!pip install opendatasets

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow_federated

In [ ]:
!nvidia-smi

In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import opendatasets as od
from tensorflow import keras
import numpy as np

In [ ]:
od.download("https://www.kaggle.com/prashant268/chest-xray-covid19-pneumonia")
od.download("https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset")

In [ ]:
LEARNING_RATE = 0.001 # 0.0001
MAX_EPOCHS = 25
TARGET_FOLDER = "weights"

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose([transforms.Resize((244, 244))
                                , transforms.ToTensor()]
                               #, transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # find mean and std of dataset
                              )

test_set = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/test', transform=transform)

train_set = dataset = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/train', transform=transform)

In [ ]:
def label_preparation(labels):
    labels = np.array(labels)
    labels[labels > 0] = 1
    return list(labels)

def label_preparation_tensor(labels):
    labels[labels > 0] = 1
    return labels

train_set.targets = label_preparation(train_set.targets)

test_set.targets = label_preparation(test_set.targets)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

In [ ]:
def calc_accuracy(result, labels):
    result = torch.sigmoid(result).round()
    
    correct_results_sum = (result == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc *= 100
    
    return acc


In [ ]:
def train(model, data_loader, optimizer, loss):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    # use pos weights because of unbalanced data set
    accuracy = 0
    for step, [images, labels] in enumerate(data_loader, 1):
        images = images.to(device)
        labels = label_preparation_tensor(labels.to(device))

        optimizer.zero_grad()
        
        result = model(images)
        targets = labels.unsqueeze(1).float()

        loss_value = loss(result.float(), targets)

        # backpropagation
        loss_value.backward()
        optimizer.step()
                                    
        if step % 10 == 0:
            accuracy += calc_accuracy(result, labels.unsqueeze(1))
            print(f"TRAINING - Step: {step}, loss: {loss_value}, rolling accuracy: {accuracy*10/step}")

In [ ]:
def test(model, test_loader, loss):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    
    with torch.no_grad():
        loss_value = 0
        accuracy = 0
        for step, [images, labels] in enumerate(test_loader, 1):
            images = images.to(device)
            labels = label_preparation_tensor(labels.to(device))

            result = model(images)
            targets = labels.detach().unsqueeze(1).float()

            loss_value += loss(result.detach(), targets)
            accuracy += calc_accuracy(result.detach(), labels.detach().unsqueeze(1))

        loss_value /= step
        accuracy /=  step
  
    print(f"TESTING - Loss: {loss_value}, Accuracy: {accuracy}")
    return accuracy > 93.

In [ ]:
def run_local_training():
    model = torchvision.models.resnet18(pretrained=False, num_classes=1)
    model.to(device)

    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters())

    # use pos weights because of unbalanced data set
    loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy

    # start training
    for epoch in range(MAX_EPOCHS):
        print(f"+++ EPOCH: {epoch+1} +++++++++")
        torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy (non-federated)
        train(model, train_loader, optimizer, loss)
        break
        # save interim weights
        torch.save(model.state_dict(), f'./{TARGET_FOLDER}/epoch_{epoch}.ckpt')

        if test(model, test_loader, loss) and epoch > 4:
            break

In [ ]:
run_local_training()